# YouTube Podcast Summarizer with Whisper and GPT-3.5 Turbo

In [1]:
import openai
import os
import re
from pytube import YouTube


In [2]:
os.environ['OPENAI_API_KEY'] = 'YOUR_OPENAI_API_KEY'   # paste your api key here
openai.api_key = os.getenv('OPENAI_API_KEY')

In [3]:
from openai import OpenAI
client = OpenAI()

In [13]:
def youtube_audio_downloader(link):
    if 'youtube.com' not in link:
        print('invalid youtube link')
        return False
    
    yt = YouTube(link)
    audio = yt.streams.filter(only_audio = True).first()
    print('downloading....', end = '')
    output_file = audio.download()
    if os.path.exists(output_file):
        print('done')
    else:
        print('error downloading audio file')
        return False
    
    basename = os.path.basename(output_file)
    name, extension = os.path.splitext(basename)
    audio_file = f'{name}.mp3'
    audio_file = re.sub('\s+','-',audio_file)
    os.rename(basename, audio_file)
    return audio_file
 
    
def transcribe(audio_file):
    if not os.path.exists(audio_file):
        print('audio file does not exists')
        return False
    
    with open(audio_file, 'rb') as f:
        print('starting transcribing...', end = '')
        transcript = client.audio.translations.create(model='whisper-1', file=f)
        print('done')
        
        name, extension = os.path.splitext(audio_file)
        transcript_filename = f'transcript-{name}.txt'
        with open(transcript_filename,'w') as f:
            f.write(transcript.text)
            
    return transcript_filename



def summarize(transcript_filename):
    
    
    with open(transcript_filename) as f:
        transcript = f.read()
        
    system_prompt = 'I want you to act as a life coach'
    prompt = f'''Create a summary of the following text.
    Text: {transcript}
    Add a title to the summary 
    Your summary should be informative and factual, covering the most important topic.
    Start a summary with an introductory paragraph that gives overview of the topic followed by
    Bullet points if possible and end the summary with a conclusion paragraph.
    
    '''
    print('starting to summarize')
    
    response= client.chat.completions.create(
    model= 'gpt-3.5-turbo',
        messages= [
            {'role':'system','content':system_prompt},
            {'role':'user','content': prompt}
        ],
        max_tokens = 2048,
        temperature = 1
    )
    
    print('done')
    
    r= response.choices[0].message.content
    return r


In [14]:
link = 'https://www.youtube.com/watch?v=8OHYynw7Yh4'
mp3_file = youtube_audio_downloader(link)
transcript_file= transcribe(mp3_file)
summary = summarize(transcript_file)
print('\n')
print(summary)

downloading....done
starting transcribing...done
starting to summarize
done


Title: Understanding the Relationship Between Tenacity, Willpower, and the AMCC

Introduction:
The text delves into the interconnected concepts of stress, willpower, motivation, discipline, and the role of the AMCC (anterior mid-cingulate cortex) in driving tenacity and willpower. It highlights the importance of challenging oneself and pushing past comfort zones to activate the AMCC and enhance various health metrics and cognitive functions.

Key Points:
- Stress perception and its impact on health depend on mindset and beliefs
- Voluntary exercise leads to health improvements, while forced exercise can have negative effects
- The AMCC plays a crucial role in generating tenacity and willpower for challenging tasks
- Success in resisting temptation or completing difficult tasks increases the size and activity of the AMCC
- Engaging in micro-sucks and macro-sucks, challenging tasks one does not want to do, can 